In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary


In [ ]:
# Data transformations for CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224,224)), # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

In [ ]:
# Loading CIFAR-10 datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:13<00:00, 12.8MB/s]


In [ ]:
batch_size = 128
num_classes = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = models.densenet121(pretrained=True)
# print(model.parameters())
# Freeze the early layers (feature extractor)
for param in model.parameters():
  # print(param)
  param.requires_grad = False

# Replace the classifier layer (fully connected layer)
num_ftrs = model.classifier.in_features
print(num_ftrs)
model.classifier = nn.Linear(num_ftrs, num_classes)

print(model.classifier)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 95.1MB/s]

1024
Linear(in_features=1024, out_features=10, bias=True)


In [ ]:
# move Model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Provide the input_size argument to the summary function
summary(model,(3, 224, 224))

In [ ]:

# Optimizer and loss function
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training function
def train_model(model, train_loader, test_loader, epochs):
    model.train()
    for epoch in range(epochs):
        total_running_loss = 0
        correct_train = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Loss and accuracy calculation
            total_running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct_train += preds.eq(labels).sum().item()

        # Average loss and accuracy for the training set
        avg_loss = total_running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        running_val_loss = 0
        correct_val = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Loss and accuracy calculation
                running_val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                correct_val += preds.eq(labels).sum().item()

        # Average loss and accuracy for the validation set
        val_loss = running_val_loss / len(test_loader.dataset)
        val_accuracy = correct_val / len(test_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # Print epoch summary
        print(f"Epoch [{epoch + 1}/{epochs}]")
        print(f"Training    - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy * 100:.2f}%")
        print(f"Validation  - Loss: {val_loss:.4f}, Accuracy: {val_accuracy * 100:.2f}%\n")

# Train and evaluate the model
train_model(model, train_loader, test_loader, epochs)



Epoch [1/5]
Training    - Loss: 0.8489, Accuracy: 74.04%
Validation  - Loss: 0.6140, Accuracy: 80.03%

Epoch [2/5]
Training    - Loss: 0.5502, Accuracy: 81.84%
Validation  - Loss: 0.5434, Accuracy: 81.42%

Epoch [3/5]
Training    - Loss: 0.5087, Accuracy: 82.89%
Validation  - Loss: 0.5291, Accuracy: 82.26%

Epoch [4/5]
Training    - Loss: 0.4895, Accuracy: 83.26%
Validation  - Loss: 0.5149, Accuracy: 82.31%

Epoch [5/5]
Training    - Loss: 0.4770, Accuracy: 83.65%
Validation  - Loss: 0.5253, Accuracy: 81.98%



In [ ]:
# Evaluate function
def evaluate_model(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      outputs = model(input)
      loss = criterion(outputs, label)
      test_loss += loss.item() * input.size(0)
      pred = outputs.argmax(dim=1)
      correct += pred.eq(label).sum().item()
  avg_loss = test_loss / len(test_loader.dataset)
  accuracy = correct/  len(test_loader.dataset)
  print(f'Test Loss : {avg_loss: .4f}, Test Accuracy : {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss :  0.5253, Test Accuracy : 0.8198
